In [13]:
import sys
sys.path.append('../')

In [14]:
import os, glob, random, cv2
import wandb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import albumentations as A
import segmentation_models_pytorch as smp
import model.metric as module_metric

from data_loader.dataloader import get_dataloader
from utils.data import get_datasize
from utils.visual import *
from albumentations.pytorch import transforms
from model.loss import *
from train import *
from pathlib import Path

In [15]:
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [16]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

lr = 1e-3
batch_size = 4
num_epoch = 100
refined = 'refined'

train_dir = f'./dataset/{refined}/crushed/train/'
val_dir = f'./dataset/{refined}/crushed/val/'

In [17]:
transform_train = A.Compose([
    A.Normalize(mean = 0.5, std=0.5),
    A.HorizontalFlip(),
    A.Rotate((-10, 10), p=0.5, border_mode=cv2.BORDER_REFLECT,),
    A.Resize(512, 512),
    transforms.ToTensorV2(transpose_mask=True)
])

transform_val = A.Compose([
    A.Normalize(mean = 0.5, std=0.5),
    A.Resize(512, 512),
    transforms.ToTensorV2(transpose_mask=True)
])

In [18]:
train_dataloader = get_dataloader(train_dir, transform_train, batch_size)
val_dataloader = get_dataloader(val_dir, transform_val, batch_size)

In [19]:
model = smp.Unet(encoder_name='efficientnet-b0', encoder_weights='imagenet', in_channels=3, classes=1, activation=None)
model = model.to(device)

In [20]:
criterion = FocalTverskyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=0.1, patience=5, min_lr=1e-6)
metrics = [getattr(module_metric, met) for met in ['IOUscore', 'PixelAccuracy']]

In [21]:
train_config = {}
train_config['Batch size'] = batch_size
train_config['Learning Rate'] = lr
train_config['Epochs'] = num_epoch

train_config['Loss fn'] = criterion.__class__.__name__
train_config['Optimizer'] = optimizer.__class__.__name__
train_config['LR Scheduler'] = scheduler.__class__.__name__
train_config['Metric'] = {str(idx+1) : metric for idx, metric in enumerate([metrics[i].__name__ for i in range(len(metrics))])}


In [22]:
save_dir = f"./saved/U-Net/{model.__dict__['name'].split('-')[1:][0].capitalize()}_Ver0/{refined}"

trainer = Trainer(model, criterion, metrics, optimizer, device, num_epoch, save_dir,
                                  data_loader=train_dataloader, valid_data_loader=val_dataloader,
                                  lr_scheduler=scheduler)

In [23]:
wandb.init(project='U_Net', name=f"{model.__dict__['name'].capitalize()}_{refined}_ver", config=train_config)

In [24]:
trainer.train()


Epoch : 0 | Train Loss : 0.83497 | Train P.A : 90.75% | Train IOU : 0.24143 | Val Loss : 0.82098 | Val P.A : 96.99% | Val IOU : 0.07151 | Training Time : 75.46sec

Epoch : 1 | Train Loss : 0.71411 | Train P.A : 97.65% | Train IOU : 0.37245 | Val Loss : 0.78425 | Val P.A : 93.93% | Val IOU : 0.11123 | Training Time : 73.57sec

Epoch : 2 | Train Loss : 0.69312 | Train P.A : 97.65% | Train IOU : 0.39127 | Val Loss : 0.77873 | Val P.A : 90.34% | Val IOU : 0.11555 | Training Time : 74.75sec

Epoch : 3 | Train Loss : 0.65383 | Train P.A : 97.99% | Train IOU : 0.40137 | Val Loss : 0.73227 | Val P.A : 97.45% | Val IOU : 0.13866 | Training Time : 74.20sec

Epoch : 4 | Train Loss : 0.65098 | Train P.A : 98.04% | Train IOU : 0.41992 | Val Loss : 0.67581 | Val P.A : 97.18% | Val IOU : 0.16277 | Training Time : 72.02sec

Epoch : 5 | Train Loss : 0.64741 | Train P.A : 98.04% | Train IOU : 0.40619 | Val Loss : 0.79494 | Val P.A : 97.67% | Val IOU : 0.10582 | Training Time : 72.68sec

Epoch : 6 | Tra

Train IOU,▁▄▅▅▅▅▆▅▆▆▆▆▆▇▇▇█▇█▇█▇▇▇▇█▇▇▇████▇█▇▇▇██
Train Loss,█▆▅▅▄▄▄▄▃▃▃▃▃▂▂▂▁▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁
Train P.A,▁▇▇▇▇▇██████████████████████████████████
Val IOU,▁▃▄▃▇▆▅▇▆▇▇▇▇███████████████████████████
Val Loss,█▇▅▇▃▄▄▂▃▁▃▂▂▂▂▂▂▁▂▁▂▂▁▁▁▂▁▁▁▂▂▁▂▂▁▁▁▁▂▂
Val P.A,▆▁▇▇▆▇▆▇█▇██▇███████████████████████████
Train IOU,0.54962
Train Loss,0.42381
Train P.A,98.94405
Val IOU,0.2203
Val Loss,0.60507
